<a href="https://colab.research.google.com/github/ychoi-kr/langchain-book/blob/main/chatper3/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT를 API에서 이용하기

## 3-3 입력과 출력의 길이 제한 및 요금에 영향을 주는 '토큰'

### Tokenizer와 tiktoken의 소개

In [1]:
!pip install tiktoken==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:00


In [2]:
import tiktoken

text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

23


In [3]:
text = "LLM을 사용해서 멋진 것을 만드는 것은 쉽지만, 프로덕션에서 사용할 수 있는 것을 만드는 것은 매우 어렵다."

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

47


## 3-4 Chat Completions API를 접하는 환경 준비

### 구글 코랩 노트북 생성

In [4]:
print("Hello World")

Hello World


### OpenAI API 키 준비

In [5]:
import os

os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

In [6]:
# 구글 코랩 보안 비밀에 API 키를 등록한 경우 다음 코드로 입력 가능
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## 3-5 Chat Completions API 사용해 보기

### OpenAI 라이브러리

In [7]:
!pip install openai==1.14.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


### Chat Completions API 호출

In [8]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! I'm John."}
  ]
)

print(response)

ChatCompletion(id='chatcmpl-99Dok3jo3IRn5Dp8Cm2YWOEIXrhff', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello John! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1711985094, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=10, prompt_tokens=23, total_tokens=33))


In [16]:
print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99Dok3jo3IRn5Dp8Cm2YWOEIXrhff",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello John! How can I assist you today?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1711985094,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_3bc1b5746c",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 23,
    "total_tokens": 33
  }
}


### 대화 이력을 바탕으로 한 응답 얻기

In [18]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}
    ]
)

print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99DuwbHsQ1xlKhhatiFbY9704tS1J",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes, you mentioned your name is John. How can I assist you, John?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1711985478,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_b28b39ffa8",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 47,
    "total_tokens": 64
  }
}


### 스트리밍으로 응답 얻기

In [19]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."}
  ],
  stream=True
)

for chunk in response:
  choice = chunk.choices[0]
  if choice.finish_reason is None:
    print(choice.delta.content)


Hello
 John
!
 How
 can
 I
 assist
 you
 today
?


### （칼럼）Completions API

In [20]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Hello! I'm John."
)

print(response.model_dump_json(indent=2))

{
  "id": "cmpl-99DvXkR5xU6n9asip9EE8pjQ4sQLX",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " I am a 26 year old Software Engineer based out of Chicago, IL."
    }
  ],
  "created": 1711985515,
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 6,
    "total_tokens": 22
  }
}


In [21]:
prompt = """Human: Hello! I'm John.
AI: Nice to meet you, John!
Human: Do you know my name?
AI: """

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt
)
print(response.model_dump_json(indent=2))

{
  "id": "cmpl-99Dw3sEhF7C1EGGLFckUsopkPhrmc",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "I am an AI and I do not have the capability to remember and store personal"
    }
  ],
  "created": 1711985547,
  "model": "gpt-3.5-turbo-instruct",
  "object": "text_completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 28,
    "total_tokens": 44
  }
}


## 3-6 Function calling

### Function calling의 샘플 코드

[OpenAI 공식 문서](https://platform.openai.com/docs/guides/gpt/function-calling)를 바탕으로 일부 수정한 코드입니다.

In [22]:
import json

def get_current_weather(location, unit="celsius"):
    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": "celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [23]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Tokyo",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [26]:
messages = [{"role": "user", "content": "What's the weather like in Seoul?"}]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

print(response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99Dwe7M1dv03IPqqnrRwWwNk9zdPp",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": {
          "arguments": "{\"location\":\"Seoul\"}",
          "name": "get_current_weather"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1711985584,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_b28b39ffa8",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 79,
    "total_tokens": 95
  }
}


In [27]:
response_message = response.choices[0].message

available_functions = {
    "get_current_weather": get_current_weather,
}
function_name = response_message.function_call.name
fuction_to_call = available_functions[function_name]
function_args = json.loads(response_message.function_call.arguments)

function_response = fuction_to_call(
    location=function_args.get("location"),
    unit=function_args.get("unit"),
)

print(function_response)

{"location": "Seoul", "temperature": "25", "unit": "celsius", "forecast": ["sunny", "windy"]}


In [28]:
messages.append(response_message)
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
)

In [29]:
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

print(second_response.model_dump_json(indent=2))

{
  "id": "chatcmpl-99DxjLLCfqbuRXYywWLHzw9ZsKeWj",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The weather in Seoul is currently 25°C with sunny and windy conditions.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1711985651,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "system_fingerprint": "fp_b28b39ffa8",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 71,
    "total_tokens": 86
  }
}
